1\. Write a function that converts number representation (bin<->dec<->hex)

In [1]:
import math
hex_dict = {0:'0', 1:'1', 2:'2', 3:'3', 4:'4', 5:'5', 6: '6', 7:'7', 8:'8', 9:'9', 10: 'a', 11: 'b', 12: 'c', 13: 'd', 14: 'e', 15: 'f', 16: 'g'}

def conv(num, da, a):
    num_str = str(num)
    dec = 0
    power = len(num_str)
    for digit in num_str:
        power-=1
        dec += int(digit, da)*(da**(power))
    result = ''
    while(dec):
        result = hex_dict[dec % a] + result
        dec = math.floor(dec/a)
    return result

x = 11111
print(conv(x, 2, 16))
print(conv(x, 2, 10))

1f
31


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [2]:
def to_float(word):
    sign = 1 if word[0]=='0' else -1
    bias = 127
    power = int(conv(word[1:9], 2, 10)) - bias
    mantissa = '1' + word[9:32]
    result = 0
    for digit in mantissa:
        if digit == '1':
            result += 2**power
        power -= 1
    return sign*result

prova = '1100000010110000000000000000000000'
print(to_float(prova))

-5.5


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [3]:
numo = 1e100
numu = 1
of = 0
uf = 0
while( numo*2 > numo ):
    of = numo
    numo *= 2
while( numu/2 < numu):
    uf = numu
    numu /= 2
print( "Overflow:", of, "Underflow:", uf, sep = ' ' )

Overflow: 1.0273702932711667e+308 Underflow: 5e-324


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [4]:
x,i = 1.0,0
while( x+10**(-i) != x ):
    i += 1
    
print("Precision:", 10**(-i), sep= ' ' )

Precision: 1e-16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(b_sol) there's an error of approximation in the sum of $b$ and $det$, the latter also is affected by an approximation error because it sums very large and very small numbers.

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [12]:
a, b, c = 0.001, 1000, 0.001

def roots2a(a, b, c):
    det = (b**2 - 4*a*c)**0.5
    print(det)
    return (-b + det)/(2*a), (-b - det)/(2*a)

print(roots2a(a, b, c))

def roots2b(a, b, c):
    det = (b**2 - 4*a*c)**0.5
    return 2*c/(-b-det), 2*c/(-b+det)

print(roots2b(a, b, c))

def roots2c(a, b, c):
    ac = 2*(a*c)**0.5
    det = (b+ac)**0.5*(b-ac)**0.5
    print(det)
    return (-b + det)/(2*a), (-b - det)/(2*a)

print(roots2c(a, b, c))

999.999999998
(-9.999894245993346e-07, -999999.999999)
(-1.000000000001e-06, -1000010.5755125057)
999.9999999979999
(-1.0000462680181954e-06, -999999.999999)


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [46]:
def f(x):
    return x*(x-1)

def df(x, delta):
    return f(x+delta)/delta-f(x)/delta


for i in [10**(-x) for x in range(2,15) if x%2==0]:
    print(i, df(1,i), sep='\t')

1.010000000000001
0.01	1.010000000000001
0.0001	1.0000999999998899
1e-06	1.0000009999177333
1e-08	1.0000000039225287
1e-10	1.000000082840371
1e-12	1.0000889005833413
1e-14	0.9992007221626509


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [47]:
import timeit
def f(x):
    return (1-x**2)**0.5

def integral(f, N):
    h=2/N
    partial = 0.
    y = -1
    for i in range(0,int(N)):
        partial+=f(y)
        y+=h
    return partial*h

print(integral(f,100))
N=1
time=0
while(time < 10):
    N*=10
    time = timeit.timeit('integral(f,' + str(N) + ')', number=10 ,globals=globals())
    print("N:", N, "time:", time/10, 's',sep='\t')
value=integral(f, N/10)
print("Integral in less than a second:", value, "precision:", abs(value*100-157.079632679), "%",sep='\t')

while(time < 60):
    N*=2
    time = timeit.timeit('integral(f,' + str(N) + ')', number=1 ,globals=globals())
    print("N:", N, "time:", time, 's',sep='\t')
value=integral(f, N/2)
print("Integral in less than a minute:", value, "precision:", abs(value*100-157.079632679), "%",sep='\t')



1.5691342555492493
N:	10	time:	1.2421499923220835e-05	s
N:	100	time:	0.00010026029995060525	s
N:	1000	time:	0.00046324979994096795	s
N:	10000	time:	0.004070423300072434	s
N:	100000	time:	0.02578640469982929	s
N:	1000000	time:	0.27851046489995496	s
N:	10000000	time:	2.420493887499833	s
Integral in less than a second:	1.5707963251473536	precision:	1.6426466231678205e-07	%
N:	20000000	time:	4.94761937000294	s
N:	40000000	time:	9.486654643998918	s
N:	80000000	time:	19.92467996999767	s
N:	160000000	time:	38.641534443002456	s
N:	320000000	time:	78.68312117199821	s
Integral in less than a minute:	1.5707963257475368	precision:	1.042463395606319e-07	%
